In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import sqlite3
import schedule
from datetime import date,datetime,timedelta

In [2]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}
link='https://www.sheypoor.com/%D8%A7%D8%B3%D8%AA%D8%A7%D9%86-%D8%AA%D9%87%D8%B1%D8%A7%D9%86/%D8%A7%D9%85%D9%84%D8%A7%DA%A9/%D8%AE%D8%B1%DB%8C%D8%AF-%D9%81%D8%B1%D9%88%D8%B4-%D8%AE%D8%A7%D9%86%D9%87-%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86'

In [ ]:
def InfoExtractor():
    conn = sqlite3.connect('posts.db')
    df=pd.read_sql_query('select * from posts where is_crawled=0',con=conn)
    my_dict={}
    cursor = conn.cursor()

    # Create a new table named 'users'
    cursor.execute('''CREATE TABLE IF NOT EXISTS info
                     (id INTEGER PRIMARY KEY AUTOINCREMENT,
                      Post_id int,
                      title TEXT,
                      description TEXT,
                      price Real,
                      create_time TEXT,
                      location TEXT,
                      option TEXT)''')

    conn.commit()

    for s in range(df.shape[0]):
            html_text=requests.get(df.loc[s,'link'], headers=headers)
            #time.sleep(np.random.randint(5)) # Seconds
            soup = BeautifulSoup(html_text.content,'lxml')
            if len(soup.find_all('table',class_='key-val border-less'))>1:
                title=soup.find('h1',class_='pull-left').text
                description=soup.find('p',class_='description').text
                price=soup.find('span',class_='item-price').text
                create_time=soup.find('time')['datetime']
                location=soup.find('span',class_='small-text').text
                table = soup.find_all('table',class_='key-val border-less')
                for t in table:
                    rows = t.find_all('tr')
                    for row in rows:
                        data = row.find('td').text
                        header = row.find('th').text
                        internal_marks = {header:data}
                        my_dict.update(internal_marks)
                conn.execute('INSERT INTO info (Post_id,title,description,price,create_time,location,option) values (?,?,?,?,?,?,?)',(s,title,description,price,create_time,location,str(my_dict)))
                conn.execute("UPDATE posts SET is_crawled = ? WHERE link = ?", (1, df.loc[s,'link']))
                print('page: '+str(s) +' - ' + 'Total Record: '+str(df.shape[0]) ,end="\r")
                conn.commit()
            else:
                print('error: This post has been deleted' ,end="\r")
    conn.close()
    
schedule.every().day.at("01:00").do(InfoExtractor)

while True:
    schedule.run_pending()
    time.sleep(60) # Wait for 1 minute

In [49]:
pd.read_sql_query('select * from posts',con=conn)

,link,is_crawled,Created_date
0,https://www.sheypoor.com/v/فروش-آپارتمان-196-م...,1,2023-04-14
1,https://www.sheypoor.com/v/فروش-آپارتمان-103-م...,1,2023-04-14
2,https://www.sheypoor.com/v/فروش-تخصصی-مسکن-مهر...,1,2023-04-14
3,https://www.sheypoor.com/v/فرمانیه-120-متر-2-خ...,1,2023-04-14
4,https://www.sheypoor.com/v/فروش-آپارتمان-135-م...,1,2023-04-14
...,...,...,...
4182,https://www.sheypoor.com/v/فروش-آپارتمان-65-مت...,0,2023-04-13
4183,https://www.sheypoor.com/v/فروش-آپارتمان-65-مت...,0,2023-04-13
4184,https://www.sheypoor.com/v/آپارتمان-فروشی-فاز-...,0,2023-04-13
4185,https://www.sheypoor.com/v/فروشنده-خریدار-ملک-...,0,2023-04-13
